In [1]:
import nrrd
from PIL import Image
import numpy as np
import json
import requests

In [2]:
SLICE_X = True
SLICE_Y = False
SLICE_Z = False
data_dir = '/mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_4/'
model_path = data_dir+'files_2/model_2_epoch.h5'
input_path ='/mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_2/'
CT_path = input_path+'case_01/case_01_IMG_CT.nrrd'
LABEL_dict = {
    "background": 0,
    "A_Carotid_L": 1,
    "A_Carotid_R": 2,
    "Arytenoid": 3,
    "Bone_Mandible": 4,
    "Brainstem": 5,
    "BuccalMucosa": 6,
    "Cavity_Oral": 7,
    "Cochlea_L": 8,
    "Cochlea_R": 9,
    "Cricopharyngeus": 10,
    "Esophagus_S": 11,
    "Eye_AL": 12,
    "Eye_AR": 13,
    "Eye_PL": 14,
    "Eye_PR": 15,
    "Glnd_Lacrimal_L": 16,
    "Glnd_Lacrimal_R": 17,
    "Glnd_Submand_L": 18,
    "Glnd_Submand_R": 19,
    "Glnd_Thyroid": 20,
    "Glottis": 21,
    "Larynx_SG": 22,
    "Lips": 23,
    "OpticChiasm": 24,
    "OpticNrv_L": 25,
    "OpticNrv_R": 26,
    "Parotid_L": 27,
    "Parotid_R": 28,
    "Pituitary": 29,
    "SpinalCord": 30,
}

In [3]:
def fixSlice(slice):
    pre_width, pre_height = slice.shape
    slice = np.flipud(slice)
    image = Image.fromarray(slice) 
    new_size = max(slice.shape[0], slice.shape[1])
    width, height = 512, 512  # Desired crop dimensions
    image = image.resize((new_size, new_size), Image.NEAREST)
    left = (image.width - width) // 2
    top = (image.height - height) // 2
    right = left + width
    bottom = top + height
    image = image.crop((left, top, right, bottom))
    slice = np.array(image)
    slice = slice/255.0
    slice = np.stack((slice,)*3, axis=-1)  ## (H, W, 3)
    slice = slice.astype(np.float32)
    return slice, pre_width, pre_height

In [4]:
def sliceVolumeImage(vol):
    (dimz, dimy, dimx) = vol.shape
    sliced_data = []
    if SLICE_X:
        for i in range(dimx):
            img, pre_width, pre_height = fixSlice(vol[:, :, i])
            sliced_data.append((img, pre_width, pre_height))
    if SLICE_Y:
        for i in range(dimy):
            img, pre_width, pre_height = fixSlice(vol[:, i, :])
            sliced_data.append((img, pre_width, pre_height))
    if SLICE_Z:
        for i in range(dimz):
            img, pre_width, pre_height = fixSlice(vol[i, :, :])
            sliced_data.append((img, pre_width, pre_height))
    return sliced_data

In [5]:
def restoreSlice(slice, width, height):
    slice = np.flipud(slice)
    resize_shape = max(width, height)
    pad_width = ((resize_shape - slice.shape[0]) // 2, (resize_shape - slice.shape[1]) // 2)
    slice = np.pad(slice, pad_width=pad_width, mode='constant', constant_values=0)
    image = Image.fromarray(slice) 
    image = image.resize((width, height), Image.NEAREST)
    slice = np.array(image)
    #slice = slice*255.0
    #slice = slice.astype(np.uint8)
    return slice

In [6]:
def restoreVolumeImage(slices, width, height, depth):
    volume = []
    if SLICE_X:
        print(len(slices), " === ", width)
        for i in range(width):
            slice = slices[i]
            volume.append(slice)
    if SLICE_Y:   
       for i in range(height):
           slice = slices[i+width]
           volume.append(slice)
    if SLICE_Z:
        for i in range(depth):
            slice = slices[i+width+height]
            volume.append(slice)  
    return np.stack(volume, axis=-1)

In [7]:
def httprequest(image_slice):
    predict_request = json.dumps({'instances' : image_slice})
    url = "http://localhost:8501/v1/models/model1:predict"
    response = requests.post(url, data=predict_request)
    prediction = response.json()['predictions'][0]
    return prediction

In [8]:
def preprocess_case(nrrd_path, output_path):
  data, header = nrrd.read(nrrd_path)
  sliced_data = sliceVolumeImage(data)
  predictions = []
  for image_slice in sliced_data:
    image = np.expand_dims(np.array(image_slice[0]), 0).tolist()
    predicted_slice = httprequest(image)
    my_array = np.array(predicted_slice)
    predicted_slice = np.argmax(my_array, axis=-1)
    predicted_slice = predicted_slice.astype(np.int32)
    predicted_slice = restoreSlice(predicted_slice, image_slice[1], image_slice[2])
    predictions.append(predicted_slice)
  predicted_volume = restoreVolumeImage(predictions, data.shape[2], data.shape[1], data.shape[0])
  print(predicted_volume.shape)
  predicted_header = header.copy()
  nrrd.write(output_path, predicted_volume, predicted_header)
  print(f"Prediction saved to: {output_path}")
  return predicted_volume

In [9]:
predicted_volume = preprocess_case(CT_path, data_dir+'files_2/predicted.nrrd')

202  ===  202
(1024, 1024, 202)
Prediction saved to: /mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_4/files_2/predicted.nrrd


In [79]:
#model=tf.keras.models.load_model(model_path)
#tf.saved_model.save(model ,data_dir+'files_2/model')
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
#tflite_model = converter.convert()
#with open(data_dir+'files_2/model.tflite', 'wb') as f:
#    f.write(tflite_model)
#interpreter = tf.lite.Interpreter(model_path=data_dir+'files_2/model.tflite')
#interpreter.allocate_tensors()
#input_details = interpreter.get_input_details()
#output_details = interpreter.get_output_details()
#input_data = np.array(np.random.random_sample(input_details[0]['shape']), dtype=np.float32)
#interpreter.set_tensor(input_details[0]['index'], input_data)
#interpreter.invoke()
#output_data = interpreter.get_tensor(output_details[0]['index'])
#print(output_data)